In [1]:
import cv2 as cv
import numpy as np
import random
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets,transforms
import torch.nn.functional as F
from torchvision import models
import time
import os
%matplotlib inline
from torch.utils.checkpoint import checkpoint_sequential
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter, ImageDraw, ImageStat

In [2]:
dataset_dir = './dataset/'
char_lst = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
province_lst = ['赣', '辽', '吉', '黑', '蒙', '冀', '晋', '陕', '宁', '鲁', '皖', '苏', '浙', '渝', '沪', '津', '京', '川', '贵', '鄂', '桂', '闽', '青', '琼', '湘', '新', '豫', '粤', '云', '藏']
file_names = os.listdir(dataset_dir)
print(len(file_names))

img_set = []
protag_set = []
tag_set = []

for file_name in file_names:
    with Image.open(dataset_dir + file_name).convert('L').resize((440, 140),Image.BICUBIC) as im:
        img_set.append(im)
#         print(im.size)
        raw_tag = file_name[:7]
        protag = [province_lst.index(raw_tag[0])]
        tag = [char_lst.index(raw_tag[1])]
        
        for ch in raw_tag[2:]:
            tag.append(char_lst.index(ch))
        
        protag_set.append(protag)
        tag_set.append(tag)

protag_set = np.array(protag_set)
tag_set = np.array(tag_set)
print(protag_set.shape)
print(tag_set.shape)

388
(388, 1)
(388, 6)


In [3]:
args  = {
    'embedding_feature': 50,
    'vocab_size': 66,
    'num_gru_layer': 2,
    'gru_hidden_feature': 7,
    'gpu': None,
    'epochs': 250,
    'lr': 0.002,
    'batch_size': 32,
    'workers': 4,
    'seed': time.time(),
    'print_freq': 2,
    'pre':None,
#     'pre': './best_model/CPRNet_model_best.pth.tar',
    'test_freq': 5
}

In [4]:
#utils.py
def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File(fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best, task_id, filename='checkpoint.pth.tar'):
    if is_best:
        torch.save(state, './best_model/' + task_id + '_model_best.pth.tar')
        
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 100 epochs"""
    lr = args['lr'] * (0.1 ** (epoch // 100))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count  

In [5]:
class list_dataset():
    def __init__(self, root, shape=None, shuffle=True, transform=None, train=False, batch_size=1, num_workers=4):
        
        random.shuffle(root)
        
        self.nSamples = len(root)
        self.transform = transform
        self.line = root
        self.train = train
        self.shape = shape
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def __len__(self):
        return self.nSamples
    def __getitem__(self, index):
        assert index <= len(self), 'index range error'
        
        img = img_set[index]
#         print(img.size)
        img = self.transform(img)
        protag = protag_set[index]
        tag = tag_set[index]
#         print(img.size())
        
        return img, protag, tag

In [6]:
# model.py
class CPR_Net(nn.Module):
    def __init__(self):
        super(CPR_Net, self).__init__()
        self.gru = nn.GRU(440, args['gru_hidden_feature'], args['num_gru_layer'], batch_first=True, bidirectional=False)
        self.pro_fc = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Linear(140, len(province_lst)),
            nn.ReLU(inplace=True),
            nn.Linear(len(province_lst), len(province_lst)),
        )
        self.lic_fc1 = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Linear(140, len(char_lst)),
            nn.ReLU(inplace=True),
            nn.Linear(len(char_lst), len(char_lst)),
        )
        self.lic_fc2 = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Linear(140, len(char_lst)),
            nn.ReLU(inplace=True),
            nn.Linear(len(char_lst), len(char_lst)),
        )
        self.lic_fc3 = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Linear(140, len(char_lst)),
            nn.ReLU(inplace=True),
            nn.Linear(len(char_lst), len(char_lst)),
        )
        self.lic_fc4 = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Linear(140, len(char_lst)),
            nn.ReLU(inplace=True),
            nn.Linear(len(char_lst), len(char_lst)),
        )
        self.lic_fc5 = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Linear(140, len(char_lst)),
            nn.ReLU(inplace=True),
            nn.Linear(len(char_lst), len(char_lst)),
        )
        self.lic_fc6 = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Linear(140, len(char_lst)),
            nn.ReLU(inplace=True),
            nn.Linear(len(char_lst), len(char_lst)),
        )
        
    def forward(self, img):
        img = img.squeeze(1)
        gru_feature, _ = self.gru(img)
        raw_province_feature = gru_feature[:,:,0]
        raw_license_feature = gru_feature[:,:,1:]
        province_feature = F.softmax(self.pro_fc(raw_province_feature), dim=1)
        
        license_feature = F.softmax(self.lic_fc1(raw_license_feature[:,:,0]), dim=1).unsqueeze(1)
        license_feature = torch.cat((license_feature, F.softmax(self.lic_fc2(raw_license_feature[:,:,1]), dim=1).unsqueeze(1)), 1)
        license_feature = torch.cat((license_feature, F.softmax(self.lic_fc3(raw_license_feature[:,:,2]), dim=1).unsqueeze(1)), 1)
        license_feature = torch.cat((license_feature, F.softmax(self.lic_fc4(raw_license_feature[:,:,3]), dim=1).unsqueeze(1)), 1)
        license_feature = torch.cat((license_feature, F.softmax(self.lic_fc5(raw_license_feature[:,:,4]), dim=1).unsqueeze(1)), 1)
        license_feature = torch.cat((license_feature, F.softmax(self.lic_fc6(raw_license_feature[:,:,5]), dim=1).unsqueeze(1)), 1)
        
        return province_feature, license_feature

In [7]:
model = CPR_Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_func_protag = torch.nn.CrossEntropyLoss()
loss_func_tags = [torch.nn.CrossEntropyLoss(), torch.nn.CrossEntropyLoss(),torch.nn.CrossEntropyLoss(),torch.nn.CrossEntropyLoss(),torch.nn.CrossEntropyLoss(),torch.nn.CrossEntropyLoss()]

In [8]:
len(list(model.parameters()))

50

In [9]:
def train(train_list, model, optimizer, epoch):
    losses = AverageMeter()
    batch_time = AverageMeter()
    
    if epoch != 0:
        adjust_learning_rate(optimizer, epoch)
    
    train_loader = torch.utils.data.DataLoader(
        list_dataset(train_list,
                        shuffle=True,
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        ]), 
                        train=True, 
                        batch_size=1,
                        num_workers=args['workers']),
        batch_size=args['batch_size'],
        drop_last=True)
    
    model.train()
    end = time.time()
    

    for i, (img, protag, tag) in enumerate(train_loader):
        img = img.type(torch.FloatTensor)
        protag = protag.type(torch.LongTensor).squeeze()
        tag = tag.type(torch.LongTensor)
        
        province_feature, license_feature = model(img)
        loss_protag = loss_func_protag(province_feature, protag)
        for kk in range(6):
            if kk == 0:
                loss_tag = loss_func_tags[kk](license_feature[:, kk], tag[:, kk])
            else:
                loss_tag += loss_func_tags[kk](license_feature[:, kk], tag[:, kk])
        
        loss = loss_protag + loss_tag
        
        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()   
        
        batch_time.update(time.time() - end)
        end = time.time()
            
        if i % args['print_freq'] == 0:
            print(loss_protag)
            print(loss_tag)
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  .format(epoch, i, len(train_loader), batch_time=batch_time, loss=losses))

In [10]:
if args['pre']:
    if os.path.isfile(args['pre']):
        print("=> loading checkpoint '{}'".format(args['pre']))
        checkpoint = torch.load(args['pre'])
        args['start_epoch'] = checkpoint['epoch']
        best_acc = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
                .format(args['pre'], checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args['pre']))

In [11]:
train_lst = [i for i in range(len(file_names))]

for epoch in range(args["epochs"]):
    train(train_lst, model, optimizer, epoch)
    if (epoch+1) % 10 == 0:
        save_checkpoint({
        'epoch': epoch + 1,
        'arch': args['pre'],
        'state_dict': model.state_dict(),
        'best_prec1': 1,
        'optimizer' : optimizer.state_dict(),
    }, 1,str('CPRNet'))
    print('Model saved!')

tensor(3.4010, grad_fn=<NllLossBackward>)
tensor(21.5000, grad_fn=<AddBackward0>)
Epoch: [0][0/12]	Time 0.157 (0.157)	Loss 24.9009 (24.9009)	
tensor(3.4001, grad_fn=<NllLossBackward>)
tensor(21.4865, grad_fn=<AddBackward0>)
Epoch: [0][2/12]	Time 0.088 (0.116)	Loss 24.8865 (24.8937)	
tensor(3.4002, grad_fn=<NllLossBackward>)
tensor(21.4792, grad_fn=<AddBackward0>)
Epoch: [0][4/12]	Time 0.100 (0.111)	Loss 24.8794 (24.8891)	
tensor(3.3992, grad_fn=<NllLossBackward>)
tensor(21.4560, grad_fn=<AddBackward0>)
Epoch: [0][6/12]	Time 0.099 (0.108)	Loss 24.8553 (24.8823)	
tensor(3.3990, grad_fn=<NllLossBackward>)
tensor(21.4383, grad_fn=<AddBackward0>)
Epoch: [0][8/12]	Time 0.188 (0.120)	Loss 24.8373 (24.8763)	
tensor(3.4015, grad_fn=<NllLossBackward>)
tensor(21.4100, grad_fn=<AddBackward0>)
Epoch: [0][10/12]	Time 0.138 (0.125)	Loss 24.8115 (24.8642)	
Model saved!
tensor(3.3937, grad_fn=<NllLossBackward>)
tensor(21.3634, grad_fn=<AddBackward0>)
Epoch: [1][0/12]	Time 0.151 (0.151)	Loss 24.7571 (24

tensor(3.2613, grad_fn=<NllLossBackward>)
tensor(19.8705, grad_fn=<AddBackward0>)
Epoch: [9][6/12]	Time 0.080 (0.092)	Loss 23.1318 (23.3539)	
tensor(3.3222, grad_fn=<NllLossBackward>)
tensor(19.8821, grad_fn=<AddBackward0>)
Epoch: [9][8/12]	Time 0.086 (0.089)	Loss 23.2043 (23.3600)	
tensor(3.3421, grad_fn=<NllLossBackward>)
tensor(19.9368, grad_fn=<AddBackward0>)
Epoch: [9][10/12]	Time 0.145 (0.095)	Loss 23.2788 (23.3074)	
Model saved!
tensor(3.2400, grad_fn=<NllLossBackward>)
tensor(19.7351, grad_fn=<AddBackward0>)
Epoch: [10][0/12]	Time 0.113 (0.113)	Loss 22.9751 (22.9751)	
tensor(3.2599, grad_fn=<NllLossBackward>)
tensor(20.0268, grad_fn=<AddBackward0>)
Epoch: [10][2/12]	Time 0.104 (0.113)	Loss 23.2868 (23.2537)	
tensor(3.2728, grad_fn=<NllLossBackward>)
tensor(20.2315, grad_fn=<AddBackward0>)
Epoch: [10][4/12]	Time 0.073 (0.100)	Loss 23.5044 (23.2956)	
tensor(3.2568, grad_fn=<NllLossBackward>)
tensor(19.7789, grad_fn=<AddBackward0>)
Epoch: [10][6/12]	Time 0.073 (0.093)	Loss 23.0357

tensor(3.1314, grad_fn=<NllLossBackward>)
tensor(19.0783, grad_fn=<AddBackward0>)
Epoch: [19][0/12]	Time 0.082 (0.082)	Loss 22.2097 (22.2097)	
tensor(3.2050, grad_fn=<NllLossBackward>)
tensor(19.5104, grad_fn=<AddBackward0>)
Epoch: [19][2/12]	Time 0.101 (0.096)	Loss 22.7154 (22.6166)	
tensor(3.1468, grad_fn=<NllLossBackward>)
tensor(19.5476, grad_fn=<AddBackward0>)
Epoch: [19][4/12]	Time 0.106 (0.102)	Loss 22.6944 (22.6676)	
tensor(3.1695, grad_fn=<NllLossBackward>)
tensor(19.2943, grad_fn=<AddBackward0>)
Epoch: [19][6/12]	Time 0.112 (0.104)	Loss 22.4638 (22.6156)	
tensor(3.2258, grad_fn=<NllLossBackward>)
tensor(19.4245, grad_fn=<AddBackward0>)
Epoch: [19][8/12]	Time 0.088 (0.101)	Loss 22.6503 (22.6727)	
tensor(3.2802, grad_fn=<NllLossBackward>)
tensor(19.4846, grad_fn=<AddBackward0>)
Epoch: [19][10/12]	Time 0.100 (0.102)	Loss 22.7648 (22.6671)	
Model saved!
tensor(3.1179, grad_fn=<NllLossBackward>)
tensor(18.9700, grad_fn=<AddBackward0>)
Epoch: [20][0/12]	Time 0.073 (0.073)	Loss 22.0

tensor(3.1000, grad_fn=<NllLossBackward>)
tensor(18.8945, grad_fn=<AddBackward0>)
Epoch: [28][6/12]	Time 0.086 (0.088)	Loss 21.9945 (22.2441)	
tensor(3.1408, grad_fn=<NllLossBackward>)
tensor(19.2128, grad_fn=<AddBackward0>)
Epoch: [28][8/12]	Time 0.078 (0.088)	Loss 22.3536 (22.3039)	
tensor(3.1758, grad_fn=<NllLossBackward>)
tensor(19.2210, grad_fn=<AddBackward0>)
Epoch: [28][10/12]	Time 0.088 (0.089)	Loss 22.3968 (22.2872)	
Model saved!
tensor(3.0882, grad_fn=<NllLossBackward>)
tensor(18.7114, grad_fn=<AddBackward0>)
Epoch: [29][0/12]	Time 0.078 (0.078)	Loss 21.7997 (21.7997)	
tensor(3.0682, grad_fn=<NllLossBackward>)
tensor(19.0751, grad_fn=<AddBackward0>)
Epoch: [29][2/12]	Time 0.102 (0.087)	Loss 22.1433 (22.1418)	
tensor(3.0981, grad_fn=<NllLossBackward>)
tensor(19.3085, grad_fn=<AddBackward0>)
Epoch: [29][4/12]	Time 0.083 (0.086)	Loss 22.4065 (22.2353)	
tensor(3.1142, grad_fn=<NllLossBackward>)
tensor(19.0401, grad_fn=<AddBackward0>)
Epoch: [29][6/12]	Time 0.127 (0.097)	Loss 22.1

tensor(3.0209, grad_fn=<NllLossBackward>)
tensor(18.8587, grad_fn=<AddBackward0>)
Epoch: [38][2/12]	Time 0.078 (0.080)	Loss 21.8796 (21.8630)	
tensor(3.0098, grad_fn=<NllLossBackward>)
tensor(18.9212, grad_fn=<AddBackward0>)
Epoch: [38][4/12]	Time 0.073 (0.079)	Loss 21.9310 (21.9275)	
tensor(3.0725, grad_fn=<NllLossBackward>)
tensor(18.6132, grad_fn=<AddBackward0>)
Epoch: [38][6/12]	Time 0.081 (0.080)	Loss 21.6857 (21.8931)	
tensor(3.0653, grad_fn=<NllLossBackward>)
tensor(18.9576, grad_fn=<AddBackward0>)
Epoch: [38][8/12]	Time 0.083 (0.080)	Loss 22.0229 (21.9555)	
tensor(3.0657, grad_fn=<NllLossBackward>)
tensor(18.8445, grad_fn=<AddBackward0>)
Epoch: [38][10/12]	Time 0.077 (0.080)	Loss 21.9102 (21.9185)	
Model saved!
tensor(3.0523, grad_fn=<NllLossBackward>)
tensor(18.4016, grad_fn=<AddBackward0>)
Epoch: [39][0/12]	Time 0.080 (0.080)	Loss 21.4539 (21.4539)	
tensor(3.0197, grad_fn=<NllLossBackward>)
tensor(18.8590, grad_fn=<AddBackward0>)
Epoch: [39][2/12]	Time 0.085 (0.086)	Loss 21.8

...

tensor(2.9226, grad_fn=<NllLossBackward>)
tensor(18.4564, grad_fn=<AddBackward0>)
Epoch: [210][2/12]	Time 0.086 (0.082)	Loss 21.3790 (21.3800)	
tensor(2.9853, grad_fn=<NllLossBackward>)
tensor(18.4264, grad_fn=<AddBackward0>)
Epoch: [210][4/12]	Time 0.087 (0.083)	Loss 21.4117 (21.4053)	
tensor(3.0139, grad_fn=<NllLossBackward>)
tensor(18.2383, grad_fn=<AddBackward0>)
Epoch: [210][6/12]	Time 0.093 (0.084)	Loss 21.2521 (21.3759)	
tensor(3.0160, grad_fn=<NllLossBackward>)
tensor(18.5192, grad_fn=<AddBackward0>)
Epoch: [210][8/12]	Time 0.132 (0.092)	Loss 21.5352 (21.4288)	
tensor(3.0152, grad_fn=<NllLossBackward>)
tensor(18.3657, grad_fn=<AddBackward0>)
Epoch: [210][10/12]	Time 0.082 (0.091)	Loss 21.3809 (21.3890)	
Model saved!
tensor(3.0154, grad_fn=<NllLossBackward>)
tensor(18.0228, grad_fn=<AddBackward0>)
Epoch: [211][0/12]	Time 0.078 (0.078)	Loss 21.0383 (21.0383)	
tensor(2.9225, grad_fn=<NllLossBackward>)
tensor(18.4564, grad_fn=<AddBackward0>)
Epoch: [211][2/12]	Time 0.080 (0.081)	Lo

tensor(3.0152, grad_fn=<NllLossBackward>)
tensor(18.3656, grad_fn=<AddBackward0>)
Epoch: [219][10/12]	Time 0.071 (0.082)	Loss 21.3808 (21.3888)	
Model saved!
tensor(3.0154, grad_fn=<NllLossBackward>)
tensor(18.0227, grad_fn=<AddBackward0>)
Epoch: [220][0/12]	Time 0.069 (0.069)	Loss 21.0381 (21.0381)	
tensor(2.9225, grad_fn=<NllLossBackward>)
tensor(18.4564, grad_fn=<AddBackward0>)
Epoch: [220][2/12]	Time 0.074 (0.073)	Loss 21.3789 (21.3799)	
tensor(2.9853, grad_fn=<NllLossBackward>)
tensor(18.4263, grad_fn=<AddBackward0>)
Epoch: [220][4/12]	Time 0.073 (0.073)	Loss 21.4116 (21.4051)	
tensor(3.0139, grad_fn=<NllLossBackward>)
tensor(18.2381, grad_fn=<AddBackward0>)
Epoch: [220][6/12]	Time 0.074 (0.074)	Loss 21.2520 (21.3757)	
tensor(3.0160, grad_fn=<NllLossBackward>)
tensor(18.5189, grad_fn=<AddBackward0>)
Epoch: [220][8/12]	Time 0.072 (0.074)	Loss 21.5349 (21.4286)	
tensor(3.0152, grad_fn=<NllLossBackward>)
tensor(18.3656, grad_fn=<AddBackward0>)
Epoch: [220][10/12]	Time 0.079 (0.075)	L

tensor(2.9225, grad_fn=<NllLossBackward>)
tensor(18.4563, grad_fn=<AddBackward0>)
Epoch: [229][2/12]	Time 0.087 (0.083)	Loss 21.3788 (21.3798)	
tensor(2.9852, grad_fn=<NllLossBackward>)
tensor(18.4263, grad_fn=<AddBackward0>)
Epoch: [229][4/12]	Time 0.073 (0.079)	Loss 21.4115 (21.4050)	
tensor(3.0139, grad_fn=<NllLossBackward>)
tensor(18.2381, grad_fn=<AddBackward0>)
Epoch: [229][6/12]	Time 0.075 (0.079)	Loss 21.2519 (21.3756)	
tensor(3.0159, grad_fn=<NllLossBackward>)
tensor(18.5186, grad_fn=<AddBackward0>)
Epoch: [229][8/12]	Time 0.078 (0.079)	Loss 21.5345 (21.4284)	
tensor(3.0152, grad_fn=<NllLossBackward>)
tensor(18.3656, grad_fn=<AddBackward0>)
Epoch: [229][10/12]	Time 0.077 (0.080)	Loss 21.3808 (21.3887)	
Model saved!
tensor(3.0154, grad_fn=<NllLossBackward>)
tensor(18.0226, grad_fn=<AddBackward0>)
Epoch: [230][0/12]	Time 0.074 (0.074)	Loss 21.0380 (21.0380)	
tensor(2.9225, grad_fn=<NllLossBackward>)
tensor(18.4563, grad_fn=<AddBackward0>)
Epoch: [230][2/12]	Time 0.092 (0.085)	Lo

tensor(3.0159, grad_fn=<NllLossBackward>)
tensor(18.5182, grad_fn=<AddBackward0>)
Epoch: [238][8/12]	Time 0.083 (0.087)	Loss 21.5341 (21.4283)	
tensor(3.0152, grad_fn=<NllLossBackward>)
tensor(18.3656, grad_fn=<AddBackward0>)
Epoch: [238][10/12]	Time 0.085 (0.087)	Loss 21.3807 (21.3885)	
Model saved!
tensor(3.0154, grad_fn=<NllLossBackward>)
tensor(18.0225, grad_fn=<AddBackward0>)
Epoch: [239][0/12]	Time 0.071 (0.071)	Loss 21.0379 (21.0379)	
tensor(2.9225, grad_fn=<NllLossBackward>)
tensor(18.4562, grad_fn=<AddBackward0>)
Epoch: [239][2/12]	Time 0.106 (0.088)	Loss 21.3787 (21.3797)	
tensor(2.9852, grad_fn=<NllLossBackward>)
tensor(18.4262, grad_fn=<AddBackward0>)
Epoch: [239][4/12]	Time 0.081 (0.087)	Loss 21.4115 (21.4048)	
tensor(3.0138, grad_fn=<NllLossBackward>)
tensor(18.2380, grad_fn=<AddBackward0>)
Epoch: [239][6/12]	Time 0.090 (0.088)	Loss 21.2518 (21.3754)	
tensor(3.0159, grad_fn=<NllLossBackward>)
tensor(18.5181, grad_fn=<AddBackward0>)
Epoch: [239][8/12]	Time 0.076 (0.086)	Lo

tensor(3.0153, grad_fn=<NllLossBackward>)
tensor(18.0224, grad_fn=<AddBackward0>)
Epoch: [248][0/12]	Time 0.153 (0.153)	Loss 21.0378 (21.0378)	
tensor(2.9225, grad_fn=<NllLossBackward>)
tensor(18.4562, grad_fn=<AddBackward0>)
Epoch: [248][2/12]	Time 0.151 (0.152)	Loss 21.3787 (21.3796)	
tensor(2.9852, grad_fn=<NllLossBackward>)
tensor(18.4262, grad_fn=<AddBackward0>)
Epoch: [248][4/12]	Time 0.118 (0.145)	Loss 21.4114 (21.4047)	
tensor(3.0138, grad_fn=<NllLossBackward>)
tensor(18.2379, grad_fn=<AddBackward0>)
Epoch: [248][6/12]	Time 0.133 (0.137)	Loss 21.2517 (21.3753)	
tensor(3.0158, grad_fn=<NllLossBackward>)
tensor(18.5172, grad_fn=<AddBackward0>)
Epoch: [248][8/12]	Time 0.090 (0.132)	Loss 21.5330 (21.4280)	
tensor(3.0151, grad_fn=<NllLossBackward>)
tensor(18.3655, grad_fn=<AddBackward0>)
Epoch: [248][10/12]	Time 0.134 (0.128)	Loss 21.3807 (21.3883)	
Model saved!
tensor(3.0153, grad_fn=<NllLossBackward>)
tensor(18.0224, grad_fn=<AddBackward0>)
Epoch: [249][0/12]	Time 0.104 (0.104)	Lo